In [38]:
# 1. 문제 정의
# 평가: f1
# target: STATUS

# 최종 파일: result.csv(컬럼 1개 pred)

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("creditcard_train.csv")
test = pd.read_csv("creditcard_test.csv")

# 3. 탐색적 데이터 분석 (EDA)
print("===== 데이터 크기 =====")
print(train.shape, test.shape)

print("\n===== 데이터 정보(자료형) =====")
print(train.info())

print("\n===== train 결측치 수 =====")
print(train.isnull().sum())

print("\n===== test 결측치 수 =====")
print(test.isnull().sum())

print("\n===== 범주형 데이터 카테고리 =====")
cols = train.select_dtypes(include = 'object').columns
for col in cols:
    set_train = set(train[col])
    set_test = set(test[col])
    same = (set_train==set_test)
    if same:
        print(col, "\t카테고리 동일함")
    else:
        print(col, "\t카테고리 동일하지 않음")

print("\n===== target 빈도 =====")
print(train['STATUS'].value_counts())

===== 데이터 크기 =====
(25519, 19) (7591, 18)

===== 데이터 정보(자료형) =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25519 entries, 0 to 25518
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   25519 non-null  int64  
 1   CODE_GENDER          25519 non-null  object 
 2   FLAG_OWN_CAR         25519 non-null  object 
 3   FLAG_OWN_REALTY      25519 non-null  object 
 4   CNT_CHILDREN         25519 non-null  int64  
 5   AMT_INCOME_TOTAL     25519 non-null  float64
 6   NAME_INCOME_TYPE     25519 non-null  object 
 7   NAME_EDUCATION_TYPE  25519 non-null  object 
 8   NAME_FAMILY_STATUS   25519 non-null  object 
 9   NAME_HOUSING_TYPE    25519 non-null  object 
 10  DAYS_BIRTH           25519 non-null  int64  
 11  DAYS_EMPLOYED        25519 non-null  int64  
 12  FLAG_MOBIL           25519 non-null  int64  
 13  FLAG_WORK_PHONE      25519 non-null  int64  
 14  FLAG_PHONE         

In [40]:
# 4. 데이터 전처리
# 결측치 처리
print("삭제 전:", train.shape)
train.dropna(subset=['OCCUPATION_TYPE'], inplace=True)
print("삭제 후:", train.shape)

target = train.pop('STATUS')

# 원-핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# 5. 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 6. 머신러닝 학습 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

from sklearn.metrics import f1_score
score = f1_score(y_val, pred)
print('\n f1:', score)

# 7. 예측 및 겨과 파일 생성
pred = rf.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv("result.csv", index=False)

삭제 전: (25519, 19)
삭제 후: (17543, 19)

 f1: 0.22018348623853212


In [43]:
# 성능 개선
# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("creditcard_train.csv")
test = pd.read_csv("creditcard_test.csv")

# 4. 데이터 전처리
# 결측치 처리
freq = train['OCCUPATION_TYPE'].mode()[0]
train['OCCUPATION_TYPE'] = train['OCCUPATION_TYPE'].fillna(freq)
test['OCCUPATION_TYPE'] = test['OCCUPATION_TYPE'].fillna(freq)
target = train.pop('STATUS')

# ID 제외
train = train.drop('ID', axis=1)
test = test.drop('ID', axis=1)

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='object').columns
for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])
# 5. 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 6. 머신러닝 학습 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200, class_weight='balanced',random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

from sklearn.metrics import f1_score
score = f1_score(y_val, pred)
print('f1:', score)

# 7. 예측 및 결과 파일 생성
pred = rf.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv("result.csv", index=False)

f1: 0.3253968253968254
